In [1]:
# 搗包
import os
import requests
from bs4 import BeautifulSoup
import urllib.parse
import time

# 挑選字符

In [4]:
字頻 = []
with open('characters_traditional_frequencies.csv', 'r',encoding='utf8') as f: # 您瞧這名字多長
    lines = f.readlines()[0:6000] # 我覺得三千差不多了，再多字都不認識了還背啥音韻地位
    for line in lines:
        字頻.append(line.split(',')[0].strip())
print(len(字頻))
字頻[0:5]
# 第一個是什麽鬼
字頻.pop(0)
字頻[0:5]

6000


['的', '是', '不', '我', '一']

# 爬取字符

In [7]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}

In [19]:
字頻[499]
# 由於測試階段曾爬取100之前，故此處從99開始。

'破'

In [18]:
# for zi in 字頻[79:100]:
for zi in 字頻[400:500]:
#     print(zi)
    zi_parse = urllib.parse.quote(zi)
    url = 'https://www.zdic.net/zd/zx/jw/' + zi_parse
    r = requests.get(url,headers=headers)
    text = r.text
    soup = BeautifulSoup(text, 'html.parser')
    try:
        jw = soup.find(name='div',attrs={"class": "zy1"})
        img_list = jw.find_all(name='img')
        
        headers['Referer'] = url

        for img in img_list[0:1]:
            # 暫時只要第一個
            src = 'https:'+ img.get('data-original')
            res = requests.get(url=src,headers=headers)
            filename = zi+'.svg'
            with open(filename,'wb') as f:
                f.write(res.content)        
    except:
        print(zi+'沒有金文')
        continue
    time.sleep(8)

線沒有金文
腦沒有金文
修沒有金文
錢沒有金文
吃沒有金文
住沒有金文
另沒有金文
錄沒有金文
換沒有金文
板沒有金文
拿沒有金文
形沒有金文
孩沒有金文
幫沒有金文
確沒有金文
候沒有金文
除沒有金文
裝沒有金文
類沒有金文
講沒有金文
案沒有金文
訴沒有金文
久沒有金文
掉沒有金文
似沒有金文
迷沒有金文
賽沒有金文
奇沒有金文
超沒有金文
運沒有金文
權沒有金文
驗沒有金文
讀沒有金文
怪沒有金文
飛沒有金文
滿沒有金文
收沒有金文
眼沒有金文
課沒有金文
破沒有金文


# 去掉水印

In [ ]:
# 將爬取的全部文件放到“去水印”文件夾中
# https://blog.csdn.net/qq_37174526/article/details/89489212
from xml.dom.minidom import parse

def 去水印(filename):
    svg = parse(filename)
    rootNode = svg.documentElement
#     print(rootNode.nodeName)
    gs = rootNode.getElementsByTagName("g")
    for g in gs:
        if g.hasAttribute('fill'):
            color = g.getAttribute('fill')
            if color[-2:] in ['fa','fb','fc']:
#                 print(color)
                rootNode.removeChild(g)

    path = rootNode.getElementsByTagName("path")
    for p in path:
        if p.hasAttribute('fill'):
            color = p.getAttribute('fill')
    #         print(color[-2:])
            if color[-2:] in ['fa','fb','fc']:
#                 print(color)
                rootNode.removeChild(p)
    # print(rootNode.toxml())
    with open(filename,'w') as f:
        f.write(rootNode.toxml())
        
## 注釋：
# 水印的位置和中英文是隨機的，但顔色是近似的，故而利用fill的顔色來判斷是否是水印
# 在瀏覽器中不可見，但是在轉換爲字體之後，會顯現出顔色（單色）
# 所以利用xml的優勢去除水印

In [ ]:
# 獲取所有文件名
# 懶得寫代碼，沒有確認文件擴展名，
# 所以需要手動刪除測試中生成的其他文件
L = []
for home, dirs, files in os.walk('去水印'): 
     for file in files:
        L.append(os.path.join(home, file))
len(L)
# https://www.cnblogs.com/frisk/p/11567175.html

In [ ]:
for l in L:
    去水印(l)

# 輸出Node.js代碼

In [ ]:
with open('create_font.js','w',encoding='utf8') as f:
    f.write('''
var fontCarrier = require('font-carrier')\n
var font = fontCarrier.create()\n
var fs = require('fs')\n
''')
    # https://stackoverflow.com/questions/29835440/meteor-referenceerror-fs-is-not-defined
    for l in L:
        char = l.split('.')[0][-1]
        jscode = "font.setSvg('" + char + "', fs.readFileSync('" + l.replace('\\','/') + "').toString())\n"
        f.write(jscode)
    f.write('''
font.output({
    path: './iconfont'
})
    ''')

# 後續操作
打開PowerShell, 安裝font-carrier后
輸入
```
node create_font.js
```
即可獲得輸出的多種格式的字體文件